In [8]:
import pandas as pd

In [9]:
countries_df = pd.read_csv("..\\data\\raw\\countries.csv", index_col="country_destination")
countries_df.head()

,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
country_destination,,,,,,
AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


In [10]:
from geopy.distance import geodesic

def distance_between_locations(country_1, country_2):
    coords_1 = (countries_df.loc[country_1, "lat_destination"], countries_df.loc[country_1, "lng_destination"])
    coords_2 = (countries_df.loc[country_2, "lat_destination"], countries_df.loc[country_2, "lng_destination"])
    return round(geodesic(coords_1, coords_2).km, 2)

In [11]:
print(distance_between_locations("AU", "CA"))

14636.98


In [12]:
train_df = pd.read_csv("..\\data\\raw\\train_users_2.csv", index_col='id')
train_df.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
id,,,,,,,,,,,,,,,
gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


We can't do user distance to each of the destinations as the training dataset doesn't have the users location, and some users have languages that aren't in our dataset so we would have to build or find our own levenshtein distance which could be difficult


Moving onto remove the anomolous ages

In [13]:
train_df[["age"]].describe()

,age
count,125461.000000
mean,49.668335
std,155.666612
min,1.000000
25%,28.000000
50%,34.000000
75%,43.000000
max,2014.000000


In [14]:
# you must be 18 years old to book on air bnb so we can remove any younger users
def remove_anomalous(df):
    return df.drop(df[(df.age < 18) | (df.age > 100)].index)

In [15]:
remove_anomalous(train_df)[["age"]].describe()

,age
count,122958.000000
mean,36.575619
std,11.681366
min,18.000000
25%,28.000000
50%,34.000000
75%,42.000000
max,100.000000


We have dropped (125461 - 122958) 2503 entries from the df


Looking at other variables in the training dataset it is unlikely that there will be any other places where we can remove anomolous data entries